In [1]:
about_the_run = input("About the run: ")
run, load_run_number = 2, 66             #  0:"single_run"  1:"keras_tuner" 2:"load_best_model"
scenario = "rcp26"
future_type= "far_future"
colab = 0
model_choice = "tf"
percentage = 100
epochs = 30
batch_size=32
limit_of_nans_in_a_timestep = 120
percentage_of_data_in_summer_months = [0.4]  # Desired number of zero values
divide_latitude_in_these_many_parts = 5  #latitude has 10 values
divide_longitude_in_these_many_parts = 6 # longitude has 14 values
climate_data_folder_name = f"{scenario}_{future_type}"

# Configurations

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import 
absolute_error
from sklearn.metrics import r2_score
import plotly.express as px
import math
import time
from sklearn.linear_model import LinearRegression
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from kerastuner.tuners import RandomSearch
import logging
import os
import sys
from contextlib import contextmanager
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import optimizers
from keras.models import load_model

C:\Users\suvam\AppData\Local\Temp\ipykernel_34168\2971726940.py:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
def get_next_folder_name(base_folder):
    run_number = 1
    while True:
        folder_name = os.path.join(base_folder, f"run_{run_number}")
        if not os.path.exists(folder_name):
            return folder_name
        run_number += 1

# Base folder directory
base_directory = r".\model_runs"

# Get the next folder name
folder_name = get_next_folder_name(base_directory)

# Create the folder
os.makedirs(folder_name, exist_ok=True)

run_number = f"run_{folder_name.split('_')[2]}"
print(run_number)

run_71


In [4]:
# Set up logging and save the log file in the folder
log_file = os.path.join(folder_name, 'log.txt')
logger = logging.getLogger(folder_name)
logger.setLevel(logging.INFO)

# Create a file handler and set the formatter
file_handler = logging.FileHandler(log_file)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

# Create a stream handler to display log messages in Jupyter Notebook console
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

# Log information
logger.info("Logging information check")
logger.info(run_number)

2024-03-20 23:41:51,555 - INFO - Logging information check
2024-03-20 23:41:51,557 - INFO - run_71


In [5]:
logger.info(f"About the run: {about_the_run}")
logger.info(f"run = {run}   =>  0:single_run  1:keras_tuner 2:load_best_model")
logger.info(f"model_choice ={model_choice}")
logger.info(f"percentage = {percentage}")
logger.info(f"epochs = {epochs}")
logger.info(f"batch_size = {batch_size}")
logger.info(f"percentage_of_data_in_summer_months = {percentage_of_data_in_summer_months}")
logger.info(f"divide_latitude_in_these_many_parts = {divide_latitude_in_these_many_parts}")  
logger.info(f"divide_longitude_in_these_many_parts = {divide_longitude_in_these_many_parts}") 


2024-03-20 23:41:51,583 - INFO - About the run: getting the predictions
2024-03-20 23:41:51,585 - INFO - run = 2   =>  0:single_run  1:keras_tuner 2:load_best_model
2024-03-20 23:41:51,586 - INFO - model_choice =tf
2024-03-20 23:41:51,588 - INFO - percentage = 100
2024-03-20 23:41:51,589 - INFO - epochs = 30
2024-03-20 23:41:51,590 - INFO - batch_size = 32
2024-03-20 23:41:51,592 - INFO - percentage_of_data_in_summer_months = [0.4]
2024-03-20 23:41:51,593 - INFO - divide_latitude_in_these_many_parts = 5
2024-03-20 23:41:51,594 - INFO - divide_longitude_in_these_many_parts = 6


In [6]:
import contextlib
import sys

@contextlib.contextmanager
def stdout_redirected(new_stdout):
    save_stdout = sys.stdout
    sys.stdout = new_stdout
    try:
        yield None
    finally:
        sys.stdout = save_stdout


In [7]:
if colab == 1:
    # !pip install cftime
    from google.colab import drive
    drive.mount('/content/drive')
    import cftime
    %run "/content/drive/My Drive/Colab Notebooks/main/functions.ipynb"
else: 
    %run "functions.ipynb"

# Data Loading

In [185]:
#bookmark
scenario="rcp85"
climate_data_folder_name = f"{scenario}_{future_type}"
climate_data_folder_name

'rcp85_far_future'

In [186]:
# Step 1: Load NDSI Labels
if colab !=1:
  ndsi_ds = xr.open_mfdataset(r".\cropped_data\label.nc")

if colab == 1:
  ndsi_ds = xr.open_mfdataset(f'/content/drive/My Drive/Colab Notebooks/cropped_data/label.nc')

# filtered_dates, nan_interpolation_df = get_filtered_dates_for_ndsi(percentage_limit=percentage_limit, pixel_interpolation_limit=pixel_interpolation_limit)
# ndsi_ds = ndsi_ds.sel(time=filtered_dates)
selected_dates = get_dates(ndsi_ds)
ndsi_labels = ndsi_ds['NDSI_Snow_Cover'].values
ndsi_ds = ndsi_ds.interpolate_na(dim='lon', method='linear',  max_gap=4, use_coordinate=False)
ndsi_ds = ndsi_ds.interpolate_na(dim='lat', method='linear', max_gap=3, use_coordinate=False)

ndsi_ds['time'] = xr.DataArray(ndsi_ds['time'].values.astype('datetime64[ns]'), dims='time', attrs=ndsi_ds['time'].attrs)
ndsi_ds.close()

In [187]:
ndsi_ds

<xarray.Dataset>
Dimensions:          (time: 6518, lon: 14, lat: 10)
Coordinates:
  * time             (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2018-12-31
  * lon              (lon) float64 10.18 10.34 10.5 10.66 ... 11.96 12.12 12.28
  * lat              (lat) float64 46.13 46.24 46.35 46.47 ... 46.92 47.03 47.14
Data variables:
    NDSI_Snow_Cover  (time, lat, lon) float32 dask.array<chunksize=(6518, 10, 14), meta=np.ndarray>

In [188]:
# Step 2: Load Data
# Load climate variables
climate_vars = ["tas", "pr", "hurs", "psl",  "rsds", "sfcWind"]
data = []

for parameter in climate_vars:
    
    if colab == 0:
      ds = xr.open_mfdataset(rf'.\cropped_data\{climate_data_folder_name}\{parameter}_{scenario}.nc') #fixxx
    if colab == 1:
      ds = xr.open_mfdataset(f'/content/drive/My Drive/Colab Notebooks/cropped_data/{climate_data_folder_name}/{parameter}_{scenario}.nc')

    data.append(ds[parameter].values)
    ds.close()

data = np.array(data)

In [189]:
ds

<xarray.Dataset>
Dimensions:  (time: 3652, y: 10, x: 14)
Coordinates:
  * time     (time) datetime64[ns] 2091-01-01T12:00:00 ... 2100-12-31T12:00:00
    lon      (y, x) float64 dask.array<chunksize=(10, 14), meta=np.ndarray>
    lat      (y, x) float64 dask.array<chunksize=(10, 14), meta=np.ndarray>
  * x        (x) float64 10.18 10.34 10.5 10.66 10.82 ... 11.8 11.96 12.12 12.28
  * y        (y) float64 46.13 46.24 46.35 46.47 ... 46.8 46.92 47.03 47.14
Data variables:
    sfcWind  (time, y, x) float32 dask.array<chunksize=(3652, 10, 14), meta=np.ndarray>

# Conversion to pandas from xarray

In [190]:
# Step 5: Flatten Data
n_time_steps = data[0].shape[0]
n_lat, n_lon = data[0].shape[1], data[0].shape[2]
logger.info("for feature")
logger.info(f"{n_time_steps} , {n_lat}, {n_lon}")

n_time_steps = ndsi_labels.shape[0]
n_lat, n_lon = ndsi_labels.shape[1], ndsi_labels.shape[2]
logger.info("for label")
logger.info(f"{n_time_steps} , {n_lat}, {n_lon}")


2024-03-18 21:29:10,534 - INFO - for feature
2024-03-18 21:29:10,535 - INFO - 3652 , 10, 14
2024-03-18 21:29:10,537 - INFO - for label
2024-03-18 21:29:10,538 - INFO - 6518 , 10, 14


In [191]:
temp0 = to_array(data, 0)
temp1 = to_array(data, 1)
temp2 = to_array(data, 2)
temp3 = to_array(data, 3)
temp4 = to_array(data, 4)
temp5 = to_array(data, 5)


In [192]:
dates = ds["time"].values
lats = ndsi_ds["lat"].values
lons = ndsi_ds["lon"].values

#********** Dates *************
dates_array_to_append = []
for a in range(len(dates)):
    x = dates[a]
    for b in range(140):
        dates_array_to_append.append(x)
dates_array_to_append = np.array(dates_array_to_append)

#********** Latitude *************
lats_array_to_append = []
for c in range(len(dates)):
    for a in range(len(lats)):
        x = lats[a]
        for b in range(len(lons)):
            lats_array_to_append.append(x)
lats_array_to_append = np.array(lats_array_to_append)

#********** Longitude *************
lons_array_to_append = []
for b in range(len(dates)*len(lats)):
    for a in range(len(lons)):
        x = lons[a]
        lons_array_to_append.append(x)

lons_array_to_append = np.array(lons_array_to_append)


logger.info(f"Length of Dates: {len(dates_array_to_append)}")
logger.info(f"Length of Latitude: {len(lats_array_to_append)}")
logger.info(f"Length of Longitude: {len(lons_array_to_append)}")

2024-03-18 21:29:13,270 - INFO - Length of Dates: 511280
2024-03-18 21:29:13,271 - INFO - Length of Latitude: 511280
2024-03-18 21:29:13,273 - INFO - Length of Longitude: 511280


# Data cleaning, feature engineering with pandas

In [193]:
dict_temp = {
    "Date": dates_array_to_append,
    "Latitude": lats_array_to_append,
    "Longitude": lons_array_to_append,
    climate_vars[0]: temp0,
    climate_vars[1]: temp1,
    climate_vars[2]: temp2,
    climate_vars[3]: temp3,
    climate_vars[4]: temp4,
    climate_vars[5]: temp5
}


# Create the DataFrame
df = pd.DataFrame(dict_temp)
df['Latitude'] = df['Latitude'].round(2)
df['Longitude'] = df['Longitude'].round(2)

df, drop_nan = df.dropna(ignore_index=False), "yes"


In [194]:
# df = df.reset_index()
df["Month"] = df['Date'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['Month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['Month']/12)

df['week_number'] = df["Date"].dt.isocalendar().week

df

,Date,Latitude,Longitude,tas,pr,hurs,psl,rsds,sfcWind,Month,month_sin,month_cos,week_number
0,2091-01-01 12:00:00,46.13,10.18,273.840942,4.659671e-04,0.004788,321.729828,10.818643,1.949361,1,5.000000e-01,0.866025,1
1,2091-01-01 12:00:00,46.13,10.34,278.132690,3.196682e-04,0.005732,308.797607,46.090126,1.519667,1,5.000000e-01,0.866025,1
2,2091-01-01 12:00:00,46.13,10.50,272.657410,4.942248e-04,0.004829,313.993256,14.193066,1.348737,1,5.000000e-01,0.866025,1
3,2091-01-01 12:00:00,46.13,10.66,272.354706,4.157453e-04,0.004551,285.309021,59.809494,3.220405,1,5.000000e-01,0.866025,1
4,2091-01-01 12:00:00,46.13,10.82,277.940155,2.112501e-04,0.005578,311.798645,27.939989,3.288927,1,5.000000e-01,0.866025,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511275,2100-12-31 12:00:00,47.14,11.64,262.393311,2.948090e-09,0.001510,201.018951,80.558182,4.071278,12,-2.449294e-16,1.000000,52
511276,2100-12-31 12:00:00,47.14,11.80,266.491943,1.702573e-08,0.001585,204.233032,80.427612,2.810014,12,-2.449294e-16,1.000000,52
511277,2100-12-31 12:00:00,47.14,11.96,263.093628,2.869060e-07,0.001359,199.822708,81.537209,1.870204,12,-2.449294e-16,1.000000,52
511278,2100-12-31 12:00:00,47.14,12.12,258.787872,2.790829e-08,0.001151,189.371887,82.408966,2.678907,12,-2.449294e-16,1.000000,52


In [195]:
lats = np.unique(df.Latitude.values)
lons = np.unique(df.Longitude.values)

latitude_bins = divide_range(lats[0], lats[-1], divide_latitude_in_these_many_parts)   #10 values 
longitude_bins = divide_range(lons[0], lons[-1], divide_longitude_in_these_many_parts) #14 values

df['Latitude_Group'] = pd.cut(df['Latitude'], bins=latitude_bins, labels=False)
df['Longitude_Group'] = pd.cut(df['Longitude'], bins=longitude_bins, labels=False)

# Apply one-hot encoding
latitude_dummies = pd.get_dummies(df['Latitude_Group'], prefix='Latitude', dtype=int)
longitude_dummies = pd.get_dummies(df['Longitude_Group'], prefix='Longitude', dtype=int)

# Concatenate one-hot encoded columns with the original DataFrame
df = pd.concat([df, latitude_dummies, longitude_dummies], axis=1)
temp = df
df = df.drop(["Latitude_Group",	"Longitude_Group"], axis =1)

temp = temp[temp["Date"] == temp.Date[1]]
grouped_latitudes = temp.groupby('Latitude_Group')['Latitude'].unique().to_dict()
logger.info("Latitude Group")
logger.info("{\n" + ",\n".join(f" {key}: {value}" for key, value in grouped_latitudes.items()) + "\n}")

grouped_longitudes = temp.groupby('Longitude_Group')['Longitude'].unique().to_dict()
logger.info("Longitude Group")
logger.info("{\n" + ",\n".join(f" {key}: {value}" for key, value in grouped_longitudes.items()) + "\n}")


2024-03-18 21:29:15,591 - INFO - Latitude Group
2024-03-18 21:29:15,593 - INFO - {
 0: [46.13 46.24],
 1: [46.35 46.47],
 2: [46.58 46.69],
 3: [46.8  46.92],
 4: [47.03 47.14]
}
2024-03-18 21:29:15,598 - INFO - Longitude Group
2024-03-18 21:29:15,601 - INFO - {
 0: [10.18 10.34 10.5 ],
 1: [10.66 10.82],
 2: [10.99 11.15],
 3: [11.31 11.47],
 4: [11.64 11.8 ],
 5: [11.96 12.12 12.28]
}


In [196]:

print(df.columns)

Index(['Date', 'Latitude', 'Longitude', 'tas', 'pr', 'hurs', 'psl', 'rsds',
       'sfcWind', 'Month', 'month_sin', 'month_cos', 'week_number',
       'Latitude_0', 'Latitude_1', 'Latitude_2', 'Latitude_3', 'Latitude_4',
       'Longitude_0', 'Longitude_1', 'Longitude_2', 'Longitude_3',
       'Longitude_4', 'Longitude_5'],
      dtype='object')


In [197]:
df= df.drop(["Month", "week_number"],axis =1)

In [198]:
parameter_array = ["tas", "pr", "hurs", "psl",  "rsds", "sfcWind"]
min_max_df = pd.read_csv("./min_max_of_all_parameters_2.csv")
for parameter in parameter_array:
    min_val, max_val =  get_min_max(min_max_df, parameter)
    df[parameter] = (df[parameter] - min_val) / (max_val - min_val)

In [199]:
df_insurance = df

In [200]:
# b_array = [0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]  # Desired number of zero values
zero_values_array = []
b_array = percentage_of_data_in_summer_months
for b in b_array:
    
    df = df_insurance
    df["Month"] = df['Date'].dt.month
    df_to_delete = df
    df = df[df['Month'].isin([1, 2, 3, 12])]
    logger.info(f"Number of datapoints in Original df: {len(df_to_delete)}")

    # Step 6: Display the final value of b
    logger.info(f"Value of b: {b}")


    # Now you can use this final value of b to sample your DataFrame
    final_sampled_data = pd.concat([df_to_delete[(df_to_delete['Date'].dt.month == month)].sample(frac=b, random_state=1) for month in range(4, 12)])
    df = pd.concat([final_sampled_data, df], ignore_index=False)
    df = df.sort_index()
    logger.info(f"Number of datapoints after sampling df: {len(df)}")


2024-03-18 21:29:19,893 - INFO - Number of datapoints in Original df: 511280
2024-03-18 21:29:19,895 - INFO - Value of b: 0.4


2024-03-18 21:29:20,553 - INFO - Number of datapoints after sampling df: 306320


In [201]:
if percentage == 100:
    df.to_csv(f"./ig_complete_df_{scenario}_{future_type}.csv")

#bookmark2

# Data splitting

In [14]:
future_type="_far_future"          #options: "", "_mid_future", "_far_future"
percentage = 100
load_run_number = 66

In [15]:
scenario_array = ["rcp26", "rcp45", "rcp85"]
# scenario_array = ["rcp85"]
prediction_array = []

for scenario in scenario_array:
    df = pd.read_csv(f"./ig_stuff/ig_complete_df_{scenario}{future_type}.csv")
    df = df.sample(frac = percentage/100, random_state=1)
    df = df.sort_index()
    logger.info(f"Number of rows: {len(df)}")

    X_og= df.drop(["Date", "Latitude", "Longitude", "Month", "Unnamed: 0"], axis=1).values
    temp_shape = X_og.shape[1]
    X = X_og.reshape(-1, 1, temp_shape)  # Reshaping to (146246, 1, 32)
    logger.info(f"X shape is: {X.shape}")

    # Check for GPU availability
    if tf.config.list_physical_devices('GPU'):
        logger.info('GPU found. Running on GPU.')
    else:
        logger.info('No GPU found. Running on CPU.')

    # Specify GPU device if available
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)

    # Convert data to TensorFlow tensors
    X_tensor = tf.convert_to_tensor(X, dtype=tf.float32)
    model = load_model(rf'.\model_runs\run_{load_run_number}\model.h5')

    y = model.predict(X_tensor)
    remove_eta_lines(run_number)
    prediction_array.append(y)

prediction_array = np.array(prediction_array)

2024-03-20 23:51:46,244 - INFO - Number of rows: 306320
2024-03-20 23:51:46,315 - INFO - X shape is: (306320, 1, 19)
2024-03-20 23:51:46,317 - INFO - GPU found. Running on GPU.


9573/9573 [==============================] - 55s 6ms/step


2024-03-20 23:52:49,124 - INFO - Number of rows: 306320
2024-03-20 23:52:49,164 - INFO - X shape is: (306320, 1, 19)
2024-03-20 23:52:49,166 - INFO - GPU found. Running on GPU.


9573/9573 [==============================] - 74s 7ms/step


2024-03-20 23:54:13,126 - INFO - Number of rows: 306320
2024-03-20 23:54:13,182 - INFO - X shape is: (306320, 1, 19)
2024-03-20 23:54:13,185 - INFO - GPU found. Running on GPU.


9573/9573 [==============================] - 81s 8ms/step


# Plotting

In [16]:
# y_for_df = y.flatten()
# y_val_pred = y_val_pred.flatten()

comparison_df = pd.DataFrame({
    "Date" : df["Date"].values,
    "Latitude" : df["Latitude"].values,
    "Longitude" : df["Longitude"].values,
    f"{scenario_array[0]}": prediction_array[0].flatten(),
    f"{scenario_array[1]}": prediction_array[1].flatten(),
    f"{scenario_array[2]}": prediction_array[2].flatten()
    })

# comparison_df = pd.DataFrame({
#     "Date" : df["Date"].values,
#     "Latitude" : df["Latitude"].values,
#     "Longitude" : df["Longitude"].values,
#     f"{scenario_array[0]}": prediction_array[0].flatten()
#     })

comparison_df['Date'] = pd.to_datetime(comparison_df['Date'])

# df_image_path = os.path.join(folder_name, 'val_vs_predicted.csv')
comparison_df.to_csv(f"./ig_stuff/ig{future_type}_predictions.csv")
comparison_df

,Date,Latitude,Longitude,rcp26,rcp45,rcp85
0,2091-01-01 12:00:00,46.13,10.18,0.610380,0.520540,0.580993
1,2091-01-01 12:00:00,46.13,10.34,0.419957,0.332166,0.470527
2,2091-01-01 12:00:00,46.13,10.50,0.667719,0.652049,0.576582
3,2091-01-01 12:00:00,46.13,10.66,0.691950,0.659971,0.670909
4,2091-01-01 12:00:00,46.13,10.82,0.665788,0.576921,0.562505
...,...,...,...,...,...,...
306315,2100-12-31 12:00:00,47.14,11.64,0.594722,0.586649,0.643759
306316,2100-12-31 12:00:00,47.14,11.80,0.534144,0.582518,0.555092
306317,2100-12-31 12:00:00,47.14,11.96,0.591901,0.591649,0.561324
306318,2100-12-31 12:00:00,47.14,12.12,0.611516,0.602783,0.633775


In [206]:
# comparison_df, scale = comparison_df.sort_values(by='Date'), "Original without mean"
# fig = px.scatter(comparison_df, x="Date" , y=scenario)
# fig.update_layout(title=f'{scenario} | {scale}', width = 1800, height = 400)

# # plotly_image_path = os.path.join(folder_name, 'Original_without_mean.png')
# # fig.write_image(plotly_image_path)

# fig

In [207]:
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  46.58) & (comparison_df['Longitude'] == 11.15)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Centre without mean"
fig = px.line(comparison_df_1, x="Date" , y = scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, 'Centre_without_mean.png')
fig.write_image(plotly_image_path)

fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [208]:
# top left
temp_lat =  lats[-3]
temp_lon = lons[2]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Top left"
fig = px.line(comparison_df_1, x="Date" , y = scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [209]:
# top right
temp_lat =  lats[-2]
temp_lon = lons[-2]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Top Right"
fig = px.line(comparison_df_1, x="Date" , y = scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [210]:
# bottom left
temp_lat =  lats[1]
temp_lon = lons[2]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Bottom left"
fig = px.line(comparison_df_1, x="Date" , y = scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [211]:
# bottom right
temp_lat =  lats[1]
temp_lon = lons[-4]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Bottom right"
fig = px.line(comparison_df_1, x="Date" , y = scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [212]:
comparison_df2 = comparison_df
comparison_df2['Date'] = pd.to_datetime(comparison_df2['Date'], dayfirst=True)
comparison_df2['Date'], scale = comparison_df2['Date'].dt.strftime('%d-%m-%Y'), "Average Daily"
comparison_df2= comparison_df2.groupby('Date')[scenario_array].mean().reset_index()
comparison_df2['Date'] = pd.to_datetime(comparison_df2['Date'], format='%d-%m-%Y')
comparison_df2 = comparison_df2.sort_values(by='Date')
# comparison_df2
fig = px.scatter(comparison_df2, x="Date" , y=scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)
fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [213]:
fig = px.line(comparison_df2, x="Date" , y=scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)
fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [214]:
comparison_df_3 = comparison_df
comparison_df_3['Date'] = pd.to_datetime(comparison_df_3['Date'], dayfirst=True)
comparison_df_3['Date'], scale = comparison_df_3['Date'].dt.strftime('%m-%Y'), "Average monthly"
comparison_df_3= comparison_df_3.groupby('Date')[scenario_array].mean().reset_index()
comparison_df_3['Date'] = pd.to_datetime(comparison_df_3['Date'])
comparison_df_3 = comparison_df_3.sort_values(by='Date')

fig = px.line(comparison_df_3, x="Date" , y=scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

C:\Users\suvam\AppData\Local\Temp\ipykernel_2092\1191913206.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [215]:
# Make a copy of the DataFrame
comparison_df_4 = comparison_df.copy()
comparison_df_4['Date'] = pd.to_datetime(comparison_df_4['Date'], dayfirst=True)
# Convert 'Date' column to month-year format
comparison_df_4['Date'], scale = comparison_df_4['Date'].dt.strftime('%Y'), "Average yearly"

# Group by 'Date' (year) and calculate the mean for each year
comparison_df_4 = comparison_df_4.groupby('Date')[scenario_array].mean().reset_index()

# Convert 'Date' back to datetime format
comparison_df_4['Date'] = pd.to_datetime(comparison_df_4['Date'])

# Sort DataFrame by 'Date'
comparison_df_4 = comparison_df_4.sort_values(by='Date')

# Create the plot with Plotly
fig = px.line(comparison_df_4, x="Date", y=scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width=1800, height=400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig.show()

C:\Users\suvam\AppData\Local\Temp\ipykernel_2092\624421656.py:3: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

